In [48]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold, GridSearchCV

In [11]:
# rows = [1757,3809,511,3798,625,3303,4095,1283,4209,1696,3511,816,245,1383,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443,3697,235,1382,4384,3418,4396,921,3176,650]
# train.loc[rows,['target']+cols+new_timeseries_columns]

In [3]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 
        'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]

In [4]:
train_leak = pd.read_csv('../Leak/train_leak.csv')
column_names = []
for i in np.arange(38):
    column_names.append('leaked_target_'+str(i))
train_leak = train_leak[column_names+['compiled_leak']]

In [6]:
# train_leak = pd.read_csv('../Leak/train_leak.csv')
train = pd.read_csv('./train.csv')
target_columns = list(train_leak.columns)

In [7]:
# Accuracy of Jiazhen's result with best_lag cutoff
print('Accuracy of Jiazhen\'s result with best_lag cutoff: {}'.format(sum(train_leak['compiled_leak'] == train['target'])/sum(train_leak['compiled_leak'].notna()))

0.9295811518324607

In [8]:
# Try using the most frequent leaked target as compiled leak
train_leak['compiled_leak'] = np.nan
for ind,row in train_leak.iterrows():
    if sum(row.notna()) != 0:
        train_leak.loc[ind,'compiled_leak'] = row[row.notna()].value_counts().index[0]

# Accuracy of using the most frequent leaks as compiled leaks
print('Accuracy of using the most frequent leaks as compiled leaks: {}'.format(sum(train_leak['compiled_leak'] == train['target'])/sum(train_leak['compiled_leak'].notna()))

0.9

In [9]:
# Get leaks without setting a cutoff of best lag
train_leak['compiled_leak'] = np.nan
train_leak['lag'] = np.nan
max_lag = 38

for i in np.arange(max_lag):
    c = train_leak['compiled_leak'].isna()
    leak_target = 'leaked_target_'+str(i)
    train_leak.loc[c,'compiled_leak'] = train_leak.loc[c,leak_target]
    train_leak.loc[c,'lag'] = i # The smallest lag that provides a target prediction for a leaked row
    
# Accuracy without a cutoff of best lag
print('Accuracy without a cutoff of best lag: {}'.format(sum(train_leak['compiled_leak'] == train['target'])/sum(train_leak['compiled_leak'].notna()))

0.9295811518324607

Setting a best-lag cutoff gives higher accuracy than using the most frequent leaked target or without the cutoff. It indicates predictions by leak with large lags have bad accuracy. Is it possible to use machine learning algorithms to predict reliability of leaked rows?

### Use machine learning algorithms to predict reliability of leaked rows

In [10]:
# Concatenate train dataframe and train_leak dataframe
train_combined = pd.concat([train,train_leak],axis=1)
train_leak_train = train_combined.loc[train_leak['compiled_leak'].notna(),cols+target_columns+['ID','target','lag']]

Four features are picked out that may have connections to reliability of leaked rows:
1. The smallest lag that provides a prediction of target
2. Number of nonzero values in leaked columns
3. Number of unique values in leaked columns, zero excluded
4. Whether the most frequent leaked target is the same as the leaked target with the smallest lag

In [11]:
train_leak_train['nonzero_count'] = 0
train_leak_train['unique_value_count'] = 0
for ind,row in train_leak_train[cols].iterrows():
    train_leak_train.loc[ind,'nonzero_count'] = sum(row != 0)
    train_leak_train.loc[ind,'unique_value_count'] = len(np.unique(row[row != 0]))

for ind,row in train_leak_train[target_columns].iterrows():
    train_leak_train.loc[ind,'most_frequent_target'] = row[row!=0].value_counts().index[0]

train_leak_train['correct_or_wrong'] = 1 # Whether a predicted target by leak is the same as the true target. This is the target for our classifier
train_leak_train.loc[(train.loc[train_leak['compiled_leak'].notna(),'target'] != train_leak_train['compiled_leak']),'correct_or_wrong'] = 0
train_leak_train['most_frequent_is_shortest_lag'] = 0
train_leak_train.loc[(train_leak_train['most_frequent_target'] == train_leak_train['compiled_leak']),'most_frequent_is_shortest_lag'] = 1

In [12]:
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',90)

In [13]:
# Train data and target for our classifier
x = train_leak_train[['lag','nonzero_count','unique_value_count','most_frequent_is_shortest_lag']]
y = train_leak_train['correct_or_wrong']

In [14]:
# Do a grid search to find best parameters for an extratrees classifier
kfold = KFold(n_splits=5,shuffle=True,random_state=1)
search_params = {
    'n_estimators': [50,100,150],
    'max_features': [1,2,3,4],
    'max_depth': [3,5,7,10],
    # 'min_samples_leaf': [2, 5,12,20],
    'min_samples_split': [2, 5, 12,20],
    'criterion': ['gini']
}

gsExTree = GridSearchCV(ExtraTreesClassifier(random_state=3,n_jobs=-1),param_grid = search_params,cv=kfold,n_jobs=4,verbose=1)
gsExTree.fit(x,y)


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   53.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed:  2.0min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=3, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': [1, 2, 3, 4], 'max_depth': [3, 5, 7, 10], 'min_samples_split': [2, 5, 12, 20], 'criterion': ['gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [16]:
print('Best cross validation score: {}'.format(gsExTree.best_score_))
print('Best parameters for the extratrees classifier:')
print(gsExTree.best_params_)
myExTree = gsExTree.best_estimator_
# myExTree = ExtraTreesClassifier(random_state=3,n_jobs=-1,criterion='gini',max_depth=5,max_features=3,min_samples_leaf=2,min_samples_split=5,n_estimators=100)

0.9568062827225131
{'criterion': 'gini', 'max_depth': 5, 'max_features': 3, 'min_samples_split': 2, 'n_estimators': 100}


The best cross validation score is almost the same as the accuracy of applying a cutoff of best_lag. Therefore, Jiazhen's idea of setting a best_lag is a simple but very efficient approach.

In [17]:
test_leak = pd.read_csv('../Leak/test_leak.csv')
test_leak = test_leak[column_names+['compiled_leak']]
test = pd.read_csv('./test.csv')
test_target_columns = list(test_leak.columns)

# Get leaks without setting the cutoff of best lag
test_leak['compiled_leak'] = np.nan
test_leak['lag'] = np.nan
max_lag = 38

for i in np.arange(max_lag):
    c = test_leak['compiled_leak'].isna()
    leak_target = 'leaked_target_'+str(i)
    test_leak.loc[c,'compiled_leak'] = test_leak.loc[c,leak_target]
    test_leak.loc[c,'lag'] = i


# Generate features for test set
test_combined = pd.concat([test,test_leak],axis=1)
test_leak_test = test_combined.loc[test_leak['compiled_leak'].notna(),cols+target_columns+['ID','lag']]

test_leak_test['nonzero_count'] = 0
test_leak_test['unique_value_count'] = 0
for ind,row in test_leak_test[cols].iterrows():
    test_leak_test.loc[ind,'nonzero_count'] = sum(row != 0)
    test_leak_test.loc[ind,'unique_value_count'] = len(np.unique(row[row != 0]))

for ind,row in test_leak_test[target_columns].iterrows():
    test_leak_test.loc[ind,'most_frequent_target'] = row[row!=0].value_counts().index[0]

test_leak_test['most_frequent_is_shortest_lag'] = 0
test_leak_test.loc[(test_leak_test['most_frequent_target'] == test_leak_test['compiled_leak']),'most_frequent_is_shortest_lag'] = 1
test_x = test_leak_test[['lag','nonzero_count','unique_value_count','most_frequent_is_shortest_lag']]

In [18]:
# Useful_test_leak.csv stores whether a leaked row is reliable. 1 indicates reliable. Otherwise 0
useful_test_leak = pd.Series(myExTree.predict(test_x))
useful_test_leak.to_csv('useful_test_leak.csv',index=False)

__Following codes were used to observe characters of unreliable leak.__

__If some assured patterns can be extracted from data, then a rule can be concluded. If some rules are not always true, try to transfer it into input for machine learning algorithms. Algorithms will evaluate it for you.__ 

In [41]:
right_prediction = train_combined.loc[(train_leak['compiled_leak'].notna()) & (train_leak['compiled_leak'] == train['target']),cols+target_columns+['ID','target']]
wrong_prediction = train_combined.loc[(train_leak['compiled_leak'].notna()) & (train_leak['compiled_leak'] != train['target']),cols+target_columns+['ID','target']]

In [47]:
def get_first_true(row):
    for i in np.arange(len(row)):
        if row[i]:
            return i
        

right_prediction['zero_count'] = 0
right_prediction['unique_value_count'] = 0
right_prediction['leak_target_used'] = 0
wrong_prediction['zero_count'] = 0
wrong_prediction['unique_value_count'] = 0
wrong_prediction['leak_target_used'] = 0

for ind,row in right_prediction[cols].iterrows():
    right_prediction.loc[ind,'zero_count'] = sum(row != 0)
    right_prediction.loc[ind,'unique_value_count'] = len(np.unique(row[row != 0]))
    right_prediction.loc[ind,'leak_target_used'] = get_first_true(right_prediction.loc[ind,target_columns] == right_prediction.loc[ind,'compiled_leak'])
for ind,row in wrong_prediction[cols].iterrows():
    wrong_prediction.loc[ind,'zero_count'] = sum(row != 0)
    wrong_prediction.loc[ind,'unique_value_count'] = len(np.unique(row[row != 0]))
    wrong_prediction.loc[ind,'leak_target_used'] = get_first_true(wrong_prediction.loc[ind,target_columns] == wrong_prediction.loc[ind,'compiled_leak'])

__In order to find other leak patterns that can decide target values, following codes counts numbers of values in a column that are coincidently equal to certain target.__

In [21]:
pattern_116 = pd.read_csv('../new_leak_patterns/pattern_1166666.66.csv')

In [22]:
pattern_116

,Unnamed: 0,8e4d0fe45,6c0e0801a,02861e414,aac52d8d9,041c5d0c9,d7875bb6c,e7c0cfd0f,d48c08bda,0c9462c08,57dd44c29,a93118262,8.50E+43,db3839ab0,27461b158,32174174c,9306da53f,95742c2bf,5831f4c76,1e6306c7c,06393096a,13bdd610a,d7d314edc,9a07d7b1f,4d2671746,822e49b95,3c8a3ced0,83635fb67,1857fbccf,c4972742d,b6c0969a2,e78e3031b,36a9a8479,e79e5f72c,092271eb3,74d7f2dc3,277ef93fc,b30e932ba,8f57141ec,350473311,value_count
0,1757,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,39
1,3809,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,38
2,511,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,37
3,3798,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,36
4,625,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,35
5,3303,360800.00,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,34
6,4095,440400.00,360800.00,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,33
7,1283,5893333.34,809000.00,440400.00,360800.00,1620000.00,150666.66,540000.00,1830000.00,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,1166666.66,31
8,4209,0.00,5893333.34,809000.00,440400.00,360800.00,1620000.00,150666.66,540000.00,1830000.00

In [70]:
pattern_325 = pd.read_csv('../new_leak_patterns/pattern_3255483.88.csv')

In [71]:
pattern_325

,Unnamed: 0,64e483341,a75d400b8,4fe8154c8,29ab304b9,20604ed8f,bd8f989f1,c1b9f4e76,4824c1e90,4ead853dc,b599b0064,d26279f1a,58ed8fb53,ff65215db,402bb0761,74d7998d4,c7775aabf,9884166a7,beb7f98fd,fd99c18b5,d83a2b684,18c35d2ea,0c8063d63,400e9303d,c976a87ad,8a088af55,5f341a818,5dca793da,db147ffca,762cbd0ab,fb5a3097e,8c0a1fa32,01005e5de,47cd6e6e4,f58fb412c,a1db86e3b,50e4f96cf,f514fdb2e,7a7da3079,bb1113dbb,value_count
0,1757,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,39
1,3809,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,38
2,511,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,37
3,3798,3795142.86,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,36
4,625,714888.88,3795142.86,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,35
5,3303,3889714.28,714888.88,3795142.86,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,34
6,4095,4547500.00,3889714.28,714888.88,3795142.86,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,33
7,1283,7498000.00,3235777.78,4547500.00,3889714.28,714888.88,3795142.86,1549200.00,3225923.08,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,3255483.88,31
8,4209,9008571.42,7498000.00,3235777.78,4547500.00,3889714.28,714888.88,3795142.

In [81]:
test.loc[test[cols[0]] == 3225500.00,cols]

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98


In [32]:
cols_196 = pattern_196.columns

In [33]:
cols_196 = list(cols_196)[1:-1]

In [44]:
train.loc[[2641],cols_196]

,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,cd24eae8a,8f3740670,2b2a10857,a00adf70e,3a48a2cd2,a396ceeb9,9280f3d04,fec5eaf1a,5b943716b,22ed6dba3,5547d6e11,e222309b0,5d3b81ef8,1184df5c2,2288333b4,f39074b55,a8b721722,13ee58af1,fb387ea33,4da206d28,ea4046b8d,ef30f6be5,b85fa8b27,2155f5e16,794e93ca6,070f95c99,939f628a7,7e814a30d,a6e871369,0dc4d6c7d,bc70cbc26,aca228668
2641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
train.loc[train['f190486d6'] == 242666.66,cols]

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
4455,242666.66,0.0,37333.34,0.0,62000.0,50000.0,69000.0,0.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0


In [46]:
train.loc[train['ID'] == '9b169cee7',cols]

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,e176a204a,6619d81fc,1db387535,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
2641,37333.34,0.0,62000.0,50000.0,69000.0,0.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0,375000.0


In [82]:
useful_leak_columns = ['64e483341','9df4daa99','8c5025c23','8e4d0fe45','b850c3e18','18562fc62','f190486d6']

In [89]:
leak_count = [0]*7

for ind,row in train.iterrows():
    for i in np.arange(7):
        if sum(train[useful_leak_columns[i]] == row['target']) != 0:
            leak_count[i] += 1

In [90]:
leak_count

[2487, 2708, 1363, 2826, 2225, 2202, 3599]

In [91]:
train.loc[2000:4000,'8e4d0fe45']

2000    0.000000e+00
2001    0.000000e+00
2002    0.000000e+00
2003    0.000000e+00
2004    0.000000e+00
2005    0.000000e+00
2006    0.000000e+00
2007    0.000000e+00
2008    0.000000e+00
2009    2.900000e+04
2010    0.000000e+00
2011    0.000000e+00
2012    0.000000e+00
2013    4.000000e+07
2014    0.000000e+00
2015    0.000000e+00
2016    0.000000e+00
2017    0.000000e+00
2018    0.000000e+00
2019    0.000000e+00
2020    0.000000e+00
2021    2.000000e+07
2022    0.000000e+00
2023    6.000000e+03
2024    0.000000e+00
2025    0.000000e+00
2026    0.000000e+00
2027    0.000000e+00
2028    0.000000e+00
2029    0.000000e+00
2030    0.000000e+00
2031    0.000000e+00
2032    0.000000e+00
2033    0.000000e+00
2034    0.000000e+00
2035    0.000000e+00
2036    0.000000e+00
2037    0.000000e+00
2038    0.000000e+00
2039    0.000000e+00
2040    0.000000e+00
2041    0.000000e+00
2042    0.000000e+00
2043    0.000000e+00
2044    0.000000e+00
2045    0.000000e+00
2046    2.850000e+05
2047    0.000

In [101]:
sum(train['f190486d6'] != 0)

1544

In [3]:
train_leak = pd.read_csv('../Leak/train_leak.csv')
train = pd.read_csv('./train.csv')

In [4]:
train_new = train.loc[train_leak['compiled_leak'].isna()]

In [41]:
count = [[0]*2]*train_new.shape[1]
count = pd.DataFrame(count, index=train_new.columns, columns=['matched_target','nonzero_cnt'])
for column in train_new.columns[2:]:
    d1 = train_new[['target']].rename(columns={'target':'key'})
    d2 = train_new[[column]].rename(columns={column:'key'})
    d2['tag'] = 1
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d1 = d1.merge(d3, how='left', on='key').fillna(0)
    count.loc[column,'matched_target'] = sum(d1['tag']==1)
    count.loc[column,'nonzero_cnt'] = sum(train_new[column] != 0)

In [43]:
pd.set_option('display.max_rows',5000)
count[(count['matched_target']>200) & (count['nonzero_cnt']>50)]

,matched_target,nonzero_cnt
a75d400b8,251,68
c1b9f4e76,202,58
9df4daa99,210,57
7e814a30d,210,52
5dca793da,208,71
fb5a3097e,275,81
5f341a818,226,76
d26279f1a,201,63
fd99c18b5,274,71
29ab304b9,228,60


In [39]:
sum(train['f190486d6'] != 0)

1544